#### Code Agents with `smolagents 🤗`
Multiple research papers have shown that having the LLM write its actions (the tool calls) in code is much better than the current standard format for tool calling, which is across the industry different shades of “writing actions as a JSON of tools names and arguments to use”.

Why is code better? Well, because we crafted our code languages specifically to be great at expressing actions performed by a computer. If JSON snippets was a better way, this package would have been written in JSON snippets and the devil would be laughing at us.

Code is just a better way to express actions on a computer. It has better:

- `Composability`: could you nest JSON actions within each other, or define a set of JSON actions to re-use later, the same way you could just define a python function?
  
- `Object management`: how do you store the output of an action like generate_image in JSON?
  
- `Generality`: code is built to express simply anything you can do have a computer do.

- `Representation in LLM training corpuses`: why not leverage this benediction of the sky that plenty of quality actions have already been included in LLM training corpuses?

This is illustrated on the figure below, taken from Executable Code Actions Elicit Better LLM Agents.

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/code_vs_json_actions.png" alt="alt text" width="800"/>



In [1]:
# Default Setup Cell.
# It imports environment variables, define 'devtools.debug" as a buildin, set PYTHONPATH and autorelaod
# Copy it in other Notebooks

import builtins

from devtools import debug
from dotenv import load_dotenv

setattr(builtins, "debug", debug)
load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f

!export PYTHONPATH=":./python"

In [2]:
from typing import Optional

from IPython.display import Markdown, display
from langchain.agents import load_tools
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import (
    TOOL_CALLING_SYSTEM_PROMPT,
    CodeAgent,
    DuckDuckGoSearchTool,
    LiteLLMModel,
    PythonInterpreterTool,
    Tool,
    ToolCallingAgent,
    ToolCollection,
    VisitWebpageTool,
    tool,
)

from python.ai_core.llm import LlmFactory

MODEL_ID = "gpt_4omini_openai"
model_name = LlmFactory(llm_id=MODEL_ID).get_litellm_model_name()
llm = LiteLLMModel(model_id=model_name)

/home/tcl/.cache/pypoetry/virtualenvs/genai-blueprint-2X6HL8i2-py3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
2025-01-07 16:09:49.315 | INFO     | python.config:yaml_file_config:47 - load /home/tcl/prj/genai-blueprint/app_conf.yaml
2025-01-07 16:09:49.320 | INFO     | python.config:yaml_file_config:56 - Override config from env. variable: edc_local


### Simple  `smolagents 🤗` CodeAgent

...  but quite impressive !

In [3]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=llm)
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many seconds would it take for a leopard at full speed to run through Pont des Arts?                        │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 distance_info = web_search(query="Pont des Arts distance")                                                  │
│   2 leopard_speed_info = web_search(query="average speed of a leopard in km/h")                                 │
│   3 print(distance_info)                                                                                        │
│   4 print(leopard_speed_info)                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Execution logs:
## Search Results

[Pont des Arts - Wikipedia](https://en.wikipedia.org/wiki/Pont_des_Arts)
The Pont des Arts (French pronunciation: [pɔ̃ dez‿aʁ]) or Passerelle des Arts ([pasʁɛl-]) is a pedestrian bridge in
Paris which crosses the River Seine. It links the Institut de France and the central square ( cour carrée ) of the 
Palais du Louvre , (which had been termed the "Palais des Arts" under the First French Empire ).

[Pont des Arts (Bridge of Arts), Paris - 
GPSmyCity](https://www.gpsmycity.com/attractions/pont-des-arts-(bridge-of-arts)-6234.html)
Pont des Arts (Bridge of Arts) in Paris, France - sight map, attraction information, photo and list of walking 
tours containing this attraction. Get offline map and directions using our GPSmyCity self-guided walking tours app 
for your mobile device. ... Travel Distance: 5.1 Km or 3.2 Miles

[A Complete Guide to the Pont des Arts in Paris - 
TripSavvy](https://www.tripsavvy.com/a-complete-guide-to-the-pont-des-arts-in-paris-4584846)
The Pont des Arts is a relative newcomer to the Parisian landscape. Emperor Napoleon I commissioned a metallic 
pedestrian bridge in around 1802. Consisting of nine arched structures, it would be Paris' first of its kind made 
of metal — a preview, perhaps, of the modern city to come.

[Paris to Pont des Arts - 5 ways to travel via line 1 subway ... - 
Rome2rio](https://www.rome2rio.com/s/Paris/Pont-Des-Arts)
The distance between Paris and Pont des Arts is 1 km. How do I travel from Paris to Pont des Arts without a car? 
The best way to get from Paris to Pont des Arts without a car is to line 1 subway which takes 6 min and costs €2.

[Pont Des Arts, Paris: How To Reach, Best Time & Tips - 
Thrillophilia](https://www.thrillophilia.com/attractions/pont-des-arts)
Places to Eat near Pont des Arts. 1. Calife: Calife restaurant is located in 3 Quai Malaquais in Paris, at a 2 min 
walking distance from Pont des Arts. It has French cuisine serving starters, appetizers, main course and gourmet 
food for lunch and dinner.

[Pont des Arts Paris: The romantic love bridge over the Seine](https://delveintoeurope.com/pont-des-arts-paris/)
The Pont des Arts bridge is one of the most famous Paris bridges, a beautiful ornate footbridge in a prime spot 
between the Louvre and Institut de France. The views from the Pont des Arts are incomparable, looking to either end
of the bridge, down the Seine or back upstream to the Pont Neuf, the Île de la Cité and towers of Notre Dame de 
Paris.

[Pont Des Arts, Paris | Ticket Price | Timings - 
TripHobo](https://www.triphobo.com/places/paris-france/pont-des-arts)
Walking distance from Louvre, this bridge is known for its thousands of love locks that have been attached to the 
railings by amorous couples. These locks have the names of the lovers written on them and the keys are thrown into 
the river. ... Pont Des Arts Address: Pont des Arts, 75006 Paris, France ; Pont Des Arts Timing: 24-hrs ; Best time
...

[The Pont des Arts Bridge - Tourism & Holiday Guide - France 
Voyage](https://www.france-voyage.com/tourism/pont-arts-bridge-2456.htm)
The Pont des Arts Bridge Tourism, holidays & weekends guide in Paris. 8.7. 3. Book online. Virtual visit. Add to 
favourites Remove from favourites. Add to itinerary Remove from itinerary. B&B HOTEL Paris 17 Batignolles. B&B 
HOTEL Paris 17 Batignolles is located in north-west Paris, just 50 metres from Porte de Clichy Metro and RER 
stations. It ...

[Pont des Arts Map - Scenic viewpoint - Paris, Île-de-France ... - Mapcarta](https://mapcarta.com/N4551075167)
Pont des Arts is a scenic viewpoint in Paris, Île-de-France. Pont des Arts is situated close to the bar La Balle au
Bond and Place de l'Institut. Overview: Map: Directions: Satellite: Photo Map: Overview: Map: Directions: 
Satellite: Photo Map: Tap on the map to travel: Pont des Arts. Notable Places in the Area.

[Learn everything about the history of Pont des Arts in Paris](https://sainte-chapelle.co/pont-des-arts/)
Built-

[Step 0: Duration 4.92 seconds| Input tokens: 2,017 | Output tokens: 126]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer(64.29)                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: 64.29

[Step 2: Duration 1.48 seconds| Input tokens: 11,450 | Output tokens: 544]

64.29

### Simple ToolCalling Agent

In [4]:
@tool
def get_weather(location: str, celsius: Optional[bool] = False) -> str:
    """
    Get weather in the next days at given location.
    Secretly this tool does not care about the location, it hates the weather everywhere.

    Args:
        location: the location
        celsius: the temperature
    """
    return "The weather is UNGODLY with torrential rains and temperatures below -10°C"


agent = ToolCallingAgent(tools=[get_weather], model=llm)
print(agent.run("What's the weather like in Paris?"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris?                                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_weather' with arguments: {'location': 'Paris', 'celsius': True}                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: The weather is UNGODLY with torrential rains and temperatures below -10°C

[Step 0: Duration 1.12 seconds| Input tokens: 1,031 | Output tokens: 19]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The weather in Paris is UNGODLY with torrential rains  │
│ and temperatures below -10°C.'}                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The weather in Paris is UNGODLY with torrential rains and temperatures below -10°C.

[Step 1: Duration 1.02 seconds| Input tokens: 2,167 | Output tokens: 52]

The weather in Paris is UNGODLY with torrential rains and temperatures below -10°C.


### RAG System with `smolagents 🤗` and `langchain 🦜️🔗`


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from python.ai_core.embeddings import EmbeddingsFactory
from python.ai_core.vector_store import VectorStoreFactory

# Usual RAG stuff : load document, split it, and add chunks into a vectorstore
loader = TextLoader("use_case_data/other/state_of_the_union.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
db = VectorStoreFactory(id="InMemory", embeddings_factory=EmbeddingsFactory()).vector_store
_ = db.add_documents(texts)

our_retriever = db.as_retriever(k=10)


# Create RetrieverTool
class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve relevant documentation"
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, retriever, **kwargs):
        super().__init__(**kwargs)
        self.retriever = retriever

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.retriever.invoke(query)
        return "\nRetrieved documents:\n" + "".join(
            f"\n\n===== Document {i} =====\n{doc.page_content}" for i, doc in enumerate(docs)
        )


# Initialize agent
def create_rag_agent(retriever):
    retriever_tool = RetrieverTool(retriever)
    return CodeAgent(tools=[retriever_tool], model=llm, max_iterations=4, verbose=True)


agent = create_rag_agent(our_retriever)
response = agent.run("What did the president say about Ketanji Brown Jackson")
debug(response)

### Understand how it works.... 

In [ ]:
display(Markdown(agent.system_prompt_template))

#### Langchain Tool with `smolagents 🤗` Search Tool

In [86]:
# search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

# agent = CodeAgent(tools=[search_tool], model=HfApiModel())

# agent.run("How many more blocks (also denoted as layers) are in BERT base encoder compared to the encoder from the architecture proposed in Attention is All You Need?")

In [5]:
agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=llm,
    additional_authorized_imports=["requests", "markdownify", "bs4"],
    use_e2b_executor=True,
)

agent.run("Qui est l'actuel premier ministre en France ?")

Installation of ['requests', 'markdownify', 'bs4', 'pickle5'] succeeded!

Logs(stdout: [], stderr: [])

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Qui est l'actuel premier ministre en France ?                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 current_pm = search("current Prime Minister of France")                                                     │
│   2 print(current_pm)                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("François Bayrou")                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: François Bayrou

[Step 3: Duration 1.79 seconds| Input tokens: 25,118 | Output tokens: 286]

'François Bayrou'

### Self Correcting Text to SQL with `smolagents 🤗`

In [6]:
from sqlalchemy import (
    Column,
    Float,
    Integer,
    MetaData,
    String,
    Table,
    create_engine,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
    {"receipt_id": 5, "customer_name": "John Doe", "price": 100.00, "tip": 10.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

table_name = "waiters"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [7]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


In [8]:
from smolagents import tool


@tool
def sql_engine(query: str) -> str:
    """Allows you to perform SQL queries

    Args:
        query: The query to perform. This should be correct SQL."""

    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output


sql_engine.description = updated_description

In [28]:
agent = CodeAgent(tools=[sql_engine], model=llm, verbose=True)
agent.run("Can you give me the name of the client who got the most expensive receipt?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of the client who got the most expensive receipt?                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To find the name of the client who got the most expensive receipt, I will need to execute a SQL query on  
the `receipts` table to retrieve the `customer_name` associated with the maximum `price`. Specifically, I'll use a 
query that groups by `customer_name` and selects the name corresponding to the maximum price.                      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
query = """                                                                                                        
SELECT customer_name                                                                                               
FROM receipts                                                                                                      
WHERE price = (                                                                                                    
    SELECT MAX(price) FROM receipts                                                                                
)                                                                                                                  
"""                                                                                                                
most_expensive_customer = sql_engine(query=query)                                                                  
print(most_expensive_customer)                                                                                     
```<end_code>                                                                                                      

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 query = """                                                                                                 │
│   2 SELECT customer_name                                                                                        │
│   3 FROM receipts                                                                                               │
│   4 WHERE price = (                                                                                             │
│   5     SELECT MAX(price) FROM receipts                                                                         │
│   6 )                                                                                                           │
│   7 """                                                                                                         │
│   8 most_expensive_customer = sql_engine(query=query)                                                           │
│   9 print(most_expensive_customer)                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Execution logs:

('John Doe',)

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: From the observation, I can see that the customer who has the most expensive receipt is John Doe. I will  
now prepare this information as my final answer.                                                                   
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("John Doe")                                                                                           
```<end_code>                                                                                                      

╭─ Executing this code: ──────────────────────────────────────────────────────────────────────────────────────────╮
│   1 final_answer("John Doe")                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Out - Final answer: John Doe

[Step 1: Duration 2.72 seconds| Input tokens: 4,362 | Output tokens: 168]

'John Doe'